%run feature_engineering.ipynb

In [1]:
%run feature_engineering.ipynb

TypeError: slice indices must be integers or None or have an __index__ method

TypeError: slice indices must be integers or None or have an __index__ method

In [ ]:
df_hospital = df_hospital
df_portobello = df_portobello

In [ ]:
df_portobello

,TIME,BIKE STANDS,AVAILABLE BIKES,DAY OF WEEK,HOUR,MINUTE
0,2020-01-28 00:00:02,30,1.000000,0.000000,0.0,0.000000
1,2020-01-28 00:05:02,30,1.000000,0.000000,0.0,0.087719
2,2020-01-28 00:10:02,30,1.000000,0.000000,0.0,0.175439
3,2020-01-28 00:15:02,30,1.000000,0.000000,0.0,0.263158
4,2020-01-28 00:20:02,30,1.000000,0.000000,0.0,0.350877
...,...,...,...,...,...,...
12955,2020-03-12 23:35:02,30,0.800000,0.333333,1.0,0.614035
12956,2020-03-12 23:40:02,30,0.833333,0.333333,1.0,0.701754
12957,2020-03-12 23:45:02,30,0.866667,0.333333,1.0,0.789474
12958,2020-03-12 23:50:02,30,0.866667,0.333333,1.0,0.877193
